# PLY 2 LAS Converter

Converts PLY point clouds to LAS format.

LAS files can be checked at [plas.io](https://plas.io/)




Made by [Artem Konevskikh](https://aiculedssul.net/)

In [ ]:
#@title Install
#@markdown After cell finish it's execution there will 'Restart Runtime' button appear. Click on it, wait for restart and continue with the next cells.

!pip install pylas[lazrs,laszip] open3d

In [2]:
#@title Load converter functions


import os
import numpy as np
import pylas
import open3d as o3d


def write_las(filename: str, xyz_points, rgb_points=None, classes=None):
    """
    creates a .las file of the generated point clouds
    """

    assert xyz_points.shape[1] == 3, 'Input XYZ points should be Nx3 float array'
    if rgb_points is None:
        rgb_points = np.ones(xyz_points.shape).astype(np.uint8) * 255
    assert xyz_points.shape == rgb_points.shape, \
        'Input RGB colors should be Nx3 float array and have same size as input XYZ points'
    if classes is None:
        classes = np.ones((xyz_points.shape[0],)).astype(np.uint8)
    assert xyz_points.shape[0] == classes.shape[0], \
        'Input Classes should be Nx1 int array and have same length as input XYZ points'

    las = pylas.create(point_format_id=2, file_version='1.2')
    las.header.x_scale = 0.0001
    las.header.y_scale = 0.0001
    las.header.z_scale = 0.0001
    las.x = xyz_points[:, 0]
    las.y = xyz_points[:, 1]
    las.z = xyz_points[:, 2]
    las.red = rgb_points[:, 0]
    las.green = rgb_points[:, 1]
    las.blue = rgb_points[:, 2]

    las.classification = classes
    las.write(filename)


def convert(ply_file, las_file):
    pcd = o3d.io.read_point_cloud(ply_file)
    pcd.estimate_normals()
    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors)
    colors = colors*255
    write_las(las_file, points, colors.astype(np.uint8))

In [ ]:
#@title Mound Google Drive
#@markdown Connect Google Drive to load and save the converted point cloud
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@title Convert
#@markdown Input PLY file
ply_file = "/content/model.ply" #@param {type:"string"}
#@markdown Output LAS file
las_file = '/content/model.las'  #@param {type:"string"}

assert os.path.exists(ply_file), "PLY file doesn't exist"
convert(ply_file, las_file)
print(f"LAS flile saved")